<table style="border: none" align="left">
   <tr style="border: none">
      <th style="border: none"><font face="verdana" size="5" color="black"><b>Use PMML to predict Iris species with IBM Watson Machine Learning</b></th>
      <th style="border: none"><img src="https://github.com/pmservice/customer-satisfaction-prediction/blob/master/app/static/images/ml_icon_gray.png?raw=true" alt="Watson Machine Learning icon" height="40" width="40"></th>
  <tr style="border: none">
       <th style="border: none"><img src="https://github.com/pmservice/wml-sample-models/blob/master/spark/customer-satisfaction-prediction/images/users_banner_2-03.png?raw=true" width="600" alt="Icon"> </th>
   </tr>
</table>

This notebook describes how to store a sample Predictive Model Markup Language (PMML) model and how to start scoring new data. 

Some familiarity with python is helpful. This notebook uses Python 3.

You will use the **Iris** data set to predict the species of an iris flower. This data set contains measurements of the iris perianth flower. 

## Learning goals

You will learn how to:

-  Work with the Watson Machine Learning (WML) instance
-  Deploy a PMML model
-  Score data for the deployed model


## Contents

1.	[Set up the environment](#setup)
2.	[Work with the WML service instance](#instance) 
3.	[Persist, deploy and score in an IBM Cloud](#scoring)
4.	[Summary and next steps](#summary)

<a id="setup"></a>
## 1. Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a [Watson Machine Learning (WML) Service](https://console.ng.bluemix.net/catalog/services/ibm-watson-machine-learning/) instance (a free plan is offered and information about how to create the instance is [here](https://dataplatform.ibm.com/docs/content/analyze-data/wml-setup.html))
-   **Not required on Watson Studio**: Install the [watson-machine-learning-client](https://pypi.python.org/pypi?%3Aaction=search&term=watson-machine-learning-client&submit=search)

<a id="instance"></a>
## 2. Working with the WML service instance

In this section you will learn how to get details about your WML service instance.

First, install the Python API client from pypi.

In [1]:
!rm -rf $PIP_BUILD/watson-machine-learning-client

In [1]:
#Install the WML client if required.
!pip install watson-machine-learning-client --upgrade
!pip install wget

Authenticate to the WML service on IBM Cloud.

**Tip**: Authentication information (your credentials) can be found in the [Service Credentials](https://console.bluemix.net/docs/services/service_credentials.html#service_credentials) tab of the service instance that you created on IBM Cloud. <BR>If you cannot see the **instance_id** field in **Service Credentials**, click **New credential (+)** to generate new authentication information. 

**Action**: Enter your WML service instance credentials here.

In [2]:
# Enter your credentials here.
wml_credentials = {
  "url": "https://ibm-watson-ml.mybluemix.net",
  "access_key": "***",
  "username": "***",
  "password": "***",
  "instance_id": "***"
}

In [3]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

In [4]:
wml_client = WatsonMachineLearningAPIClient(wml_credentials)

Details about your WML service instance are available in `service_instance.details`.

In [5]:
# List information about your WML service instance.
print(wml_client.service_instance.details)

{'metadata': {'created_at': '2018-01-30T10:39:51.938Z', 'url': 'https://ibm-watson-ml.mybluemix.net/v3/wml_instances/fd1db635-c9f3-4d83-afc7-32efec367e5e', 'modified_at': '2018-01-30T14:16:00.601Z', 'guid': 'fd1db635-c9f3-4d83-afc7-32efec367e5e'}, 'entity': {'region': 'us-south', 'account': {'id': '8dba8395f5b8dc6fcab49d43161ab9f1', 'type': 'TRIAL', 'name': "Wojciech Sobala's Account"}, 'deployments': {'url': 'https://ibm-watson-ml.mybluemix.net/v3/wml_instances/fd1db635-c9f3-4d83-afc7-32efec367e5e/deployments'}, 'source': 'Bluemix', 'status': 'Active', 'plan_id': '0f2a3c2c-456b-40f3-9b19-726d2740b11c', 'published_models': {'url': 'https://ibm-watson-ml.mybluemix.net/v3/wml_instances/fd1db635-c9f3-4d83-afc7-32efec367e5e/published_models'}, 'organization_guid': 'a7b3ef43-5dc2-41fc-8fa7-29b0846e5807', 'plan': 'standard', 'space_guid': 'a6b628f9-3f64-45c0-bf38-739876b63fec', 'usage': {'deployment_count': {'current': 1, 'limit': 1000}, 'expiration_date': '2018-02-01T00:00:00.000Z', 'comput

Get information about the stored models for your WML instance.

In [6]:
# List information about the stored models.
wml_client.repository.list_models()

------------------------------------  ------------------------------  ------------------------  -----------------  -----
GUID                                  NAME                            CREATED                   FRAMEWORK          TYPE
51cdbdea-28ea-488d-9edd-dab2a8c44208  Sentiment Prediction            2018-01-30T14:15:06.339Z  mllib-2.0          model
b7e9fc69-f0db-444f-89b4-fe7aa5e8a4f1  Handwritten Digits Recognition  2018-01-30T14:15:24.106Z  scikit-learn-0.17  model
------------------------------------  ------------------------------  ------------------------  -----------------  -----


Get a list of the deployed models for your WML service instance.

In [7]:
# List the deployed models.
wml_client.deployments.list()

------------------------------------  ------------------  ------  ------------------------  -----------------
GUID                                  NAME                TYPE    CREATED                   FRAMEWORK
06faa2cd-e12b-4595-b96d-555d9fae6363  Digits recognition  online  2018-01-30T14:15:59.342Z  scikit-learn-0.17
------------------------------------  ------------------  ------  ------------------------  -----------------


<a id="scoring"></a>
## 3. Persist, deploy and score model in a Cloud

In this section you will use the Python API client for the WML service to create an online deployment and score a new data record.

For more information, see the <a href="http://wml-api-pyclient.mybluemix.net/" target="_blank" rel="noopener no referrer">Python API client documentation</a>.

- [3.1. Persist model](#persist)
- [3.2. Create the online deployment](#deploy)
- [3.3. Score data](#score)


### 3.1. Persist model<a id="persist"></a>

**Action**: Use `wget` to download the sample PMML model, `iris_chaid.xml` from the Git project.

In [8]:
# Download sample PMML model, iris_chaid.xml, from Git.
import wget, os

sample_dir = 'pmml_sample_model'
if not os.path.isdir(sample_dir):
    os.mkdir(sample_dir)
    
filename=os.path.join(sample_dir, 'iris_chaid.xml')
if not os.path.isfile(filename):
    filename = wget('https://github.com/pmservice/wml-sample-models/raw/master/pmml/iris-species/model/iris_chaid.xml', out=sample_dir)

Store the downloaded file as *CHAID PMML model for Iris data* in the WML service repository, then list all the files stored in the repository.

In [9]:
# Save the downloaded file to the WML repository.
props_pmml = {wml_client.repository.ModelMetaNames.NAME: 'CHAID PMML model for Iris data',
              wml_client.repository.ModelMetaNames.AUTHOR_NAME: "IBM", 
              wml_client.repository.ModelMetaNames.FRAMEWORK_NAME: "pmml", 
              wml_client.repository.ModelMetaNames.FRAMEWORK_VERSION: "4.2"}

model_details = wml_client.repository.store_model(filename, props_pmml)
wml_client.repository.list_models()

------------------------------------  ------------------------------  ------------------------  -----------------  -----
GUID                                  NAME                            CREATED                   FRAMEWORK          TYPE
51cdbdea-28ea-488d-9edd-dab2a8c44208  Sentiment Prediction            2018-01-30T14:15:06.339Z  mllib-2.0          model
b7e9fc69-f0db-444f-89b4-fe7aa5e8a4f1  Handwritten Digits Recognition  2018-01-30T14:15:24.106Z  scikit-learn-0.17  model
fae8e498-136e-4c35-81bd-a9a69a50b1c7  CHAID PMML model for Iris data  2018-01-30T14:17:19.921Z  pmml-4.2           model
------------------------------------  ------------------------------  ------------------------  -----------------  -----


**Note:** From the list of downloaded files, you can see that model is successfully stored in the WML service repository.

### 3.2. Create the online deployment<a id="deploy"></a>

Now create the online deployment, *Iris species prediction*, for the stored model, then list all the online deployments for the model.

In [10]:
# Create the online deployment.
model_uid = wml_client.repository.get_model_uid(model_details)
model_deployment = wml_client.deployments.create(model_uid, name = 'Iris species prediction')

wml_client.deployments.list()

------------------------------------  -----------------------  ------  ------------------------  -----------------
GUID                                  NAME                     TYPE    CREATED                   FRAMEWORK
06faa2cd-e12b-4595-b96d-555d9fae6363  Digits recognition       online  2018-01-30T14:15:59.342Z  scikit-learn-0.17
56f3501f-844c-4cb7-9bbc-02131da8f6bf  Iris species prediction  online  2018-01-30T14:17:22.742Z  pmml-4.2
------------------------------------  -----------------------  ------  ------------------------  -----------------


**Note:** From the list of deployed models, you can see that model was  successfully created.

### 3.3. Score data<a id="score"></a>

Send a sample record using the Python API client to check that the online scoring endpoint is available.

In [11]:
# Verify the online scoring endpoint.
model_scoring_url = wml_client.deployments.get_scoring_url(model_deployment)

print(model_scoring_url)

https://ibm-watson-ml.mybluemix.net/v3/wml_instances/fd1db635-c9f3-4d83-afc7-32efec367e5e/published_models/fae8e498-136e-4c35-81bd-a9a69a50b1c7/deployments/56f3501f-844c-4cb7-9bbc-02131da8f6bf/online


You can see that the endpoint url is available for sending scoring requests.

Now, you can score data and predict the species of iris flower.

In [12]:
# Score data and predict species of flower.
scoring_data = {'fields': ['Sepal.Length', 'Sepal.Width', 'Petal.Length', 'Petal.Width'], 'values': [[5.1, 3.5, 1.4, 0.2]]}
predictions = wml_client.deployments.score(model_scoring_url, scoring_data) 

print(predictions)

{'fields': ['PredictedValue', 'Confidence'], 'values': [['setosa', 1.0]]}


As we can see from the prediction, this is the Iris Setosa flower.

<a id="summary"></a>
## 4. Summary and next steps     

You successfully completed this notebook! You learned how to use Watson Machine Learning for PMML model deployment and scoring. 

Check out our [documentation](https://dataplatform.ibm.com/docs/content/analyze-data/wml-setup.html) for more samples, tutorials, documentation, how-tos, and blog posts. 

### Citations

Dua, D. and Karra Taniskidou, E. (2017). [UCI Machine Learning Repository](http://archive.ics.uci.edu/ml). Irvine, CA: University of California, School of Information and Computer Science.


### Authors

**Wojciech Sobala** is a Data Scientist in IBM.

Copyright © 2018 IBM. This notebook and its source code are released under the terms of the MIT License.

<div style="background:#F5F7FA; height:110px; padding: 2em; font-size:14px;">
<span style="font-size:18px;color:#152935;">Love this notebook? </span>
<span style="font-size:15px;color:#152935;float:right;margin-right:40px;">Don't have an account yet?</span><br>
<span style="color:#5A6872;">Share it with your colleagues and help them discover the power of Watson Studio!</span>
<span style="border: 1px solid #3d70b2;padding:8px;float:right;margin-right:40px; color:#3d70b2;"><a href="https://ibm.co/wsnotebooks" target="_blank" style="color: #3d70b2;text-decoration: none;">Sign Up</a></span><br>
</div>